
# 🧭 Goal-Seeded Model-based Planning in a Spatial Grid

This Colab notebook implements a simulation based on the following formulation:

> _Consider an agent navigating a $32 \times 32$ grid world, where the raw context $\Psi \in \mathbb{R}^{1024}$ encodes the full environment state (e.g., obstacle map, sensory input). The agent's goal is to infer a path to a destination using an internal latent representation $\Phi \in \mathbb{R}^d$ with $d \ll 1024$, e.g., a low-dimensional code encoding goal location and path direction. The inference loop proceeds via simulation and refinement._

We show how **half-cycle right inference** can help the agent reach a consistent prediction of map state by leveraging amortized recognition and goal-seeded simulation.


In [1]:

import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import random
from tqdm import trange


In [2]:

GRID_SIZE = 32

def generate_environment():
    env = np.zeros((GRID_SIZE, GRID_SIZE))
    # Add random obstacles
    for _ in range(100):
        x, y = random.randint(0, GRID_SIZE-1), random.randint(0, GRID_SIZE-1)
        env[x, y] = 1
    return env

env = generate_environment()
plt.imshow(env, cmap='gray')
plt.title("Obstacle Map")
plt.show()


In [3]:

class GoalEncoder(nn.Module):
    def __init__(self, latent_dim=8):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(2, 32),
            nn.ReLU(),
            nn.Linear(32, latent_dim)
        )
    def forward(self, goal):
        return self.fc(goal)

class TrajectoryDecoder(nn.Module):
    def __init__(self, latent_dim=8):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(latent_dim, 256),
            nn.ReLU(),
            nn.Linear(256, GRID_SIZE * GRID_SIZE),
        )
    def forward(self, z):
        return torch.sigmoid(self.fc(z)).view(-1, GRID_SIZE, GRID_SIZE)

class InverseModel(nn.Module):
    def __init__(self, latent_dim=8):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Flatten(),
            nn.Linear(GRID_SIZE * GRID_SIZE, 256),
            nn.ReLU(),
            nn.Linear(256, latent_dim)
        )
    def forward(self, obs):
        return self.fc(obs)


In [4]:

device = 'cuda' if torch.cuda.is_available() else 'cpu'
latent_dim = 8

encoder = GoalEncoder(latent_dim).to(device)
decoder = TrajectoryDecoder(latent_dim).to(device)
inverse = InverseModel(latent_dim).to(device)

optimizer = torch.optim.Adam(list(encoder.parameters()) +
                             list(decoder.parameters()) +
                             list(inverse.parameters()), lr=1e-3)

goal = torch.tensor([[30/32, 30/32]], dtype=torch.float32).to(device)
goal_latent = encoder(goal)

for step in trange(2000):
    z = encoder(goal)
    pred_map = decoder(z)
    z_rec = inverse(pred_map)
    goal_rec = encoder(goal)

    loss = F.mse_loss(z_rec, z.detach())

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if step % 200 == 0:
        print(f"Step {step}, Loss: {loss.item():.5f}")


  1%|          | 20/2000 [00:00<00:19, 101.03it/s]

Step 0, Loss: 0.07410


 12%|█▏        | 230/2000 [00:01<00:11, 153.72it/s]

Step 200, Loss: 0.00000


 21%|██        | 413/2000 [00:03<00:15, 101.96it/s]

...

 80%|████████  | 1609/2000 [00:15<00:03, 103.36it/s]

Step 1600, Loss: 0.00011


 91%|█████████ | 1812/2000 [00:18<00:02, 69.11it/s]

Step 1800, Loss: 0.00000


100%|██████████| 2000/2000 [00:20<00:00, 98.25it/s] 


In [5]:

with torch.no_grad():
    goal = torch.tensor([[30/32, 30/32]], dtype=torch.float32).to(device)
    z = encoder(goal)
    decoded_map = decoder(z).cpu().numpy()[0]

plt.figure(figsize=(6,6))
plt.imshow(decoded_map, cmap='hot')
plt.title("Predicted Environment from Goal Latent")
plt.colorbar()
plt.show()


In [6]:
def compute_obstacle_similarity(obstacle_map, predicted_map):
    """
    Computes IoU, precision, and recall between true obstacle map and predicted map.
    Maps must be binary (1 for obstacle, 0 for free space).
    """
    obstacle_map = obstacle_map.astype(bool)
    predicted_map = predicted_map.astype(bool)

    intersection = np.logical_and(obstacle_map, predicted_map).sum()
    union = np.logical_or(obstacle_map, predicted_map).sum()

    iou = intersection / union if union > 0 else 1.0
    precision = intersection / predicted_map.sum() if predicted_map.sum() > 0 else 1.0
    recall = intersection / obstacle_map.sum() if obstacle_map.sum() > 0 else 1.0

    return {"IoU": iou, "Precision": precision, "Recall": recall}

metrics = compute_obstacle_similarity(env, decoded_map)
print("Similarity Metrics:")
print(metrics)


Similarity Metrics:
{'IoU': np.float64(0.091796875), 'Precision': np.float64(0.091796875), 'Recall': np.float64(1.0)}
